In [1]:
import os

from openai import OpenAI

OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

client = OpenAI(api_key=OPENAI_API_KEY)

prompt = "내일 날씨는 어때?"

completion = client.chat.completions.create(
  model='gpt-3.5-turbo-0125',
  messages=[{'role': 'user', 'content': prompt}],
  temperature=0.0,
)

print(completion.choices[0].message.content)

죄송합니다. 날씨 정보는 실시간으로 업데이트되는 정보이기 때문에 정확한 내일 날씨를 알려드릴 수 없습니다. 날씨 예보 사이트나 앱을 통해 내일 날씨를 확인해보시는 것을 추천드립니다.


# 어떻게 실시간으로 부족한 데이터를 주입할까?
- Prompt에 부족한 데이터를 추가하는 방식

In [2]:
weather_info = """(종합) 오늘 낮까지 비 곳, 오늘 강풍과 풍랑 유의, 내일 늦은 오후~저녁 경기동부 소나기, 모레 낮부터 비
○ (오늘, 30일) 대체로 흐림, 서울.경기도(경기남서부 제외) 중심 낮(12~15시)까지 비 곳
○ (내일, 7월 1일) 구름많음, 대기불안정으로 늦은 오후(15~18시)부터 저녁(18~21시) 사이 경기동부 소나기 곳, 서해5도 대체로 맑다가 저녁부터 구름많아짐
○ (모레, 7월 2일) 대체로 흐림, 서울.인천.경기도 낮(12~15시)부터 비, 서해5도(연평도 부근) 늦은 밤(21~24시) 가끔 비 곳
* 예상 강수량(30일 낮까지)
- 서울.경기도(경기남서부 제외): 5mm 미만
* 소나기에 의한 예상 강수량(7월 1일 늦은 오후~저녁)
- 경기동부: 5~20mm
* 예상 강수량(7월 2일 낮~)
- 서울.인천.경기도: 10~50mm
- 서해5도(7월 2일 밤): 5~20mm"""

prompt = '내일 날씨는 어때?' + '\n\n' + weather_info

completion = client.chat.completions.create(
    model='gpt-3.5-turbo-0125',
    messages=[{'role': 'user', 'content': prompt}],
    temperature=0.0
)

print(completion.choices[0].message.content)

내일은 구름많고 대기불안정으로 늦은 오후부터 저녁 사이에 경기동부에서 소나기가 올 것으로 예상됩니다. 모레는 대체로 흐리고 서울, 인천, 경기도에서 낮부터 비가 오겠습니다. 서해5도에서는 늦은 밤에 가끔 비가 올 것으로 예상됩니다. 오늘은 대체로 흐리고 서울, 경기도 중심으로 낮까지 비가 오는 곳이 있을 것으로 보입니다. 강풍과 풍랑에 유의하시기 바랍니다.


# 어떻게 질문과 연관된 정보를 가져올까? (RAG의 핵심)
- RAG에서 Retrieval이 나오는 이유
  - Retrieval = 연관 텍스트 가져오는 행위 (검색도 Retrieval에 포함)
- 어떻게 Retrieve 할까?
  - 방법 1) 텍스트에서 특정 단어가 자주 발생 --> 해당 단어랑 연관이 있을꺼다!
  - 방법 2) 근데 자주 발생하는 단어가 항상 자주 발생하는 단어 --> 이런건 가중치를 줄이자
  - 방법 3) 딥러닝 모델한테 텍스트 간의 문서를 직접 학습시키면 어떨까?
    - 2012년 Word2Vec 논문에서 시작. 흔히 임베딩이라고 한다.
- 가장 간단한 방법 1 시연

In [3]:
def count_common_words(text1, text2):
    words1 = text1.split()
    words2 = text2.split()
    common_words = set(words1) & set(words2)
    return len(common_words)

question = '한국의 전통 음식에 대해 무엇을 알고 계신가요?'

ref_texts = [
    '한국의 전통 음식은 다양한 재료와 조리법으로 유명합니다. 김치, 불고기, 비빔밥 등이 대표적인 예시입니다.',
    '한국의 음식 문화는 건강에 좋은 재료를 사용하는 것으로 알려져 있습니다. 발효 식품인 김치가 대표적입니다.',
    '한국의 역사와 문화는 매우 흥미롭습니다. 고궁과 한복, 그리고 태권도 등이 유명한 문화 요소입니다.'
]

common_word_counts = [count_common_words(question, ref_text) for ref_text in ref_texts]

most_similar_index = common_word_counts.index(max(common_word_counts))
print(f'가장 유사한 텍스트 번호: {most_similar_index + 1}')
print(f'겹치는 단어의 수: {common_word_counts[most_similar_index]}')

가장 유사한 텍스트 번호: 1
겹치는 단어의 수: 2
